# 3. Manipulating models

Genome-scale metabolic models can be modified to simulate, for example, the effects on fluxes when a gene is knocked out or mutated.This is extremely practical and time-saving for effective strain development and increased understanding of an organism.

First, the ecoli_core_model is loaded into the jupyter notebook again using COBRApy.

In [1]:
pip install cobra

Note: you may need to restart the kernel to use updated packages.


In [2]:
from cobra.io import read_sbml_model

In [3]:
model=read_sbml_model('e_coli_core.xml')

## Making temporary changes to the model

Usually one relies on making copies if objects need to be changed but the original state needs to be retained. Unfortunately, making copies of models is time consuming.

To repeat: In this case the stoichiometric processes of the e.coli_core_model are considered, which are responsible for biomass production and this production should be increased.

The following command displays all reactions and flows that are optimised to achieve this goal:

In [4]:
model.optimize()

,fluxes,reduced_costs
PFK,7.477382,-1.387779e-17
PFL,0.000000,2.081668e-17
PGI,4.860861,0.000000e+00
PGK,-16.023526,6.938894e-18
PGL,4.959985,1.517883e-17
...,...,...
NADH16,38.534610,0.000000e+00
NADTRHD,0.000000,-2.546243e-03
NH4t,4.765319,-1.387779e-17
O2t,21.799493,0.000000e+00


In [5]:
%%time
copy_of_model=model.copy()

Wall time: 192 ms


Yes, even milliseconds add up pretty quickly if you need to run many simulation (e.g. if you need to knock out every single gene individually in the model to check if it is essential or not).

In [6]:
%%time
for gene in model.genes:
    mutant=model.copy()
    mutant.genes.get_by_id(gene.id).knock_out()

Wall time: 12.1 s


For that reason cobrapy provides a mechanism that is less time consuming. Almost all methods that make changes to the mdoel such as knocking-out genes, reactions, adding or removing metabolites, reactions etc can be automatically reverted upon exit from a python context. How this works is probably best understood by looking at an example.

In [7]:
%%time
with model:
    for gene in model.genes:
        gene.knock_out()

Wall time: 24 ms


Here, the 'with model' statements starts the context and changes done to the model one indentation level to the right, are automatically recorded. When that block finishes, the context manager is requested to roll-back all changes leaving the model looking exactly as it did before all the changes.

Changing flux bounds can as indicated also be done reversibly. For example let's set the lower and upper bound of phosphoglycerate kinase (PGK) to 0 (effectively knocking out the reaction).

In [8]:
with model:
    model.reactions.PGK.bounds = 0, 0
    print("PGK's bounds inside the with statement")
    print(model.reactions.PGK.lower_bound, model.reactions.PGK.bounds)
    print('Mutant growth rate: ', model.optimize().objective_value)
print("PGK's bounds outside the with statement")
print(model.reactions.PGK.bounds)

PGK's bounds inside the with statement
0 (0, 0)
Mutant growth rate:  -5.487661025838073e-16
PGK's bounds outside the with statement
(-1000.0, 1000.0)


## Slim versus full optimize

Mathematical solvers are now so fast that for many small to mid-size models computing the solution can be even faster than it takes us to collect the values from the solver and convert that to objects that are usable for in python. When we use model.optimize we gather values for all reactions and metabolites and that can take some time. If we are only interested in the flux value of a single reaction or the objective, it is faster to instead use model.slim_optimize which only does the optimization and returns the objective value leaving it up to you to fetch other values that you may need. For example, let's optimize and get the flux value of the ATPM reaction.

In [9]:
%%time
solution=model.optimize()
solution.fluxes['ATPM']

Wall time: 8 ms


8.39

In [10]:
%%time
model.slim_optimize()
model.reactions.ATPM.flux

Wall time: 0 ns


8.39

## Changing the medium

The media composition can also be displayed and, if necessary, optimised in relation to the specified objective of the flux balance analysis. One can access the medium condition using model.medium. The indicated bound is the effective upper uptake bound.

In [11]:
model.medium

{'EX_co2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_h_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0}

In [12]:
solution=model.optimize()
print(solution.fluxes['BIOMASS_Ecoli_core_w_GAM'])

0.8739215069684324


Changing the carbon source in the medium can be achieved by adjusting the flux bounds of the respective exchange reactions appropriately. For example, the following code block removes glucose from the medium and adds succinate.

In [13]:
medium=model.medium
with model:
    medium['EX_glc__D_e']=0
    medium['EX_succ_e']=10
    model.medium=medium
    solution=model.optimize()
    print(solution.fluxes['BIOMASS_Ecoli_core_w_GAM'])

0.3975630154277627


Changing the carbon source to succinate led to a significant drop in growth rate.

## Exercise (10 min)
Change the carbon source in the medium to a different carbon source. What is the difference in the growth rate observed? How about growing E. coli under anaerobic conditions?

In [ ]:
medium=model.medium
with model:
    medium['EX_o2_e']=0
    model.medium=medium
    solution=model.optimize()
    print(solution.fluxes['BIOMASS_Ecoli_core_w_GAM'])

## Adding reactions and pathways

To add reactions and pathways the tools "Reaction" and "Metabolite" from COBRApy are used. 

In [14]:
from cobra import Reaction, Metabolite

Ok, let's create a new reactions.

In [15]:
new_reaction=Reaction('alchemy')

This reaction is going to convert water into gold (unfortunately lead is not part of E. coli metabolism; creating wine would be blasphemy). So we need to create a new metabolite, since gold is not yet part of E. coli's native metabolism.

In [16]:
gold=Metabolite(id='gold_c', compartment='c')

Now, we're going to specify the reaction's stoichiometry.

In [17]:
new_reaction.add_metabolites({model.metabolites.h2o_c: -1, gold: 1})

Printing the reaction reveals that the reaction indeed converts water into gold.

In [18]:
print(new_reaction.build_reaction_string())

h2o_c --> gold_c


Now, let's add the new reaction to the model.

In [19]:
model.add_reactions([new_reaction])

Quickly check that the reaction was indeed added to the model.

In [20]:
model.reactions.alchemy

Reaction identifier,alchemy
Name,
Memory address,0x01271e186588
Stoichiometry,h2o_c --> gold_c H2O H2O -->
GPR,
Lower bound,0.0
Upper bound,1000.0


Let's produce some gold then!

In [21]:
model.objective=model.reactions.alchemy
model.optimize().objective_value

0.0

What happened? Forgot to add an exchange reaction so that gold can leave the system.

In [22]:
model.add_boundary(model.metabolites.gold_c, type='demand')

Reaction identifier,DM_gold_c
Name,demand
Memory address,0x01271e2f25c8
Stoichiometry,gold_c --> -->
GPR,
Lower bound,0
Upper bound,1000.0


In [23]:
model.objective=model.reactions.alchemy
model.optimize().objective_value

1000.0